# Hands-On: Building a Web Search Tool with CrewAI

Let's roll up our sleeves and build a workflow using CrewAI tools to scrape the content from the website and then perform RAG on it.  
We need to write the code that will make the tool work. Before we begin, let’s install the crewai-tools and crewai packages using pip:  

In [34]:
pip install crewai-tools crewai

Note: you may need to restart the kernel to use updated packages.


In this example, we’ll use three different tools:
- ScrapeWebsiteTool to scrape the website,
- FileWriterTool to write the file,
- and TXTSearchTool to search the context for RAG.

## Step 1: Scraping a website
- We first import the necessary libraries and initialize the ScrapeWebsiteTool. 
- This tool is used to extract content from a website.
- In this case, it’s set to scrape content from Wikipedia's "Artificial Intelligence" page:

In [5]:
from crewai_tools import ScrapeWebsiteTool, FileWriterTool, TXTSearchTool
import requests

# Initialize the tool, potentially passing the session
tool = ScrapeWebsiteTool(website_url='https://en.wikipedia.org/wiki/Artificial_intelligence')  


# Extract the text
text = tool.run()
print(text)

Using Tool: Read website content

Artificial intelligence - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page Contents Current events Random article About Wikipedia Contact us
Contribute
Help Learn to edit Community portal Recent changes Upload file Special pages
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate Create account Log in
Pages for logged out editors learn more
Contributions Talk
Contents
move to sidebar
hide
(Top)
1
Goals
Toggle Goals subsection
1.1
Reasoning and problem-solving
1.2
Knowledge representation
1.3
Planning and decision-making
1.4
Learning
1.5
Natural language processing
1.6
Perception
1.7
Social intelligence
1.8
General intelligence
2
Techniques
Toggle Techniques subsection
2.1
Search and optimization
2.1.1
State space search
2.1.2
Local search
2.2
Logic
2.3
Probabilistic methods for uncertain reasoning
2.4
Classifiers and statistical learning methods
2.5
Artificial neural networks
2.6
Deep learn

## Step2: Write the extracted text to a file
We now use the FileWriterTool to save the extracted text into a file named ai.txt. 

In [41]:
# Initialize the tool
file_writer_tool = FileWriterTool()

# Write content to a file in a specified directory
result = file_writer_tool._run(filename='ai.txt', content = text, directory = '', overwrite=True)
print(result)

Content successfully written to ai.txt


## Step 3: Set up the text search tool
We set up another tool to search the contents of the file ai.txt that we just saved. We also set the environment variable for the OpenAI API key.

In [44]:
import os
from crewai_tools import TXTSearchTool

os.environ["OPENAI_API_KEY"] = ""

# Initialize the tool with a specific text file, so the agent can search within the given text file's content
tool = TXTSearchTool(txt='ai.txt')

Inserting batches in chromadb: 100%|██████████████| 2/2 [00:02<00:00,  1.44s/it]


## Step 4: Create an agent for the task and execute it
We create a data analyst agent with the role of an educator. This agent's task is to answer the question "What is Natural Language Processing?" based on the text we searched in the file.

In [57]:
from crewai import Agent, Task, Crew

data_analyst = Agent(
    role='Educator',
    goal='Answer questions based on the content of ai.txt using the TXTSearchTool.',
    backstory='You are a data expert capable of understanding and explaining technical content.',
    verbose=True,
    allow_delegation=False,
    tools=[tool]
)

test_task = Task(
    description="Use the tool to answer: What is natural language processing?",
    tools=[tool],
    agent=data_analyst,
    expected_output='A precise and informative answer about Natural Language Processing.'
)

crew = Crew(
    agents=[data_analyst],
    tasks=[test_task]
)

output = crew.kickoff()

# Agent: Educator
## Task: Use the tool to answer: What is natural language processing?


# Agent: Educator
## Thought: I need to search for the definition and explanation of natural language processing in the content of ai.txt.
## Using tool: Search a txt's content
## Tool Input: 
"{\"search_query\": \"What is natural language processing?\"}"
## Tool Output: 
Relevant Content:
Hughes-Castleberry, Kenna, "A Murder Mystery Puzzle: The literary puzzle Cain's Jawbone , which has stumped humans for decades, reveals the limitations of natural-language-processing algorithms", Scientific American , vol. 329, no. 4 (November 2023), pp. 81–82. "This murder mystery competition has revealed that although NLP ( natural-language processing ) models are capable of incredible feats, their abilities are very much limited by the amount of context they receive. This [...] could cause [difficulties] for researchers who hope to use them to do things such as analyze ancient languages . In some cases, there